In [1]:
%load_ext autoreload
%autoreload 2
%pwd
%cd ..
%pwd

/home/julien-rsbrg/Documents/UCL/SemanticToLiking


/home/julien-rsbrg/venv/venv_semantic_to_liking/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


'/home/julien-rsbrg/Documents/UCL/SemanticToLiking'

In [2]:
from sklearn.preprocessing import StandardScaler

import torch
import torch_geometric
import numpy as np
import matplotlib.pyplot as plt


from src.models.nn_layers import MLPModel
from src.models.gnn_layers import myGATConv
from src.models.frameworks import GNN_naive_framework
import src.loading as loading
import src.processing as processing

from visualization.analyse_model import plot_errors_labels_comparison

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/home/julien-rsbrg/venv/venv_semantic_to_liking/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
## Data


data = loading.load_data()

def nor_function(a,b):
    return (a or b) and not(a and b)

data["NoExp_Exp"] = data.apply(lambda row: nor_function(row["word1_experience"]>50,row["word2_experience"]>50),axis=1)

scaler = StandardScaler()
data.loc[:,["word1_sc_liking","word2_sc_liking","sc_senenceBERT_mpnet_similarity","sc_depressionCont","sc_NoExp_Exp"]] = scaler.fit_transform(data.loc[:,["word1_liking","word2_liking","senenceBERT_mpnet_similarity","depressionCont","NoExp_Exp"]])
data.loc[:,["word1_sc_liking","word2_sc_liking","sc_senenceBERT_mpnet_similarity","sc_depressionCont","sc_NoExp_Exp"]]


== Load Data: start ==
== Load Data: end ==


,word1_sc_liking,word2_sc_liking,sc_senenceBERT_mpnet_similarity,sc_depressionCont,sc_NoExp_Exp
0,-1.172222,-1.152427,1.118413,-0.229448,-0.891164
1,-1.172222,0.605846,-0.021906,-0.229448,-0.891164
2,-1.172222,0.546244,0.867681,-0.229448,1.122128
3,-1.172222,-0.019980,1.244298,-0.229448,-0.891164
4,-1.172222,0.456840,0.193088,-0.229448,-0.891164
...,...,...,...,...,...
198235,0.456083,0.546244,0.040509,-0.708297,1.122128
198236,0.456083,1.201871,-0.668605,-0.708297,1.122128
198237,0.394638,0.546244,-1.800021,-0.708297,1.122128
198238,0.394638,1.201871,-0.165669,-0.708297,1.122128


In [4]:
subdata = data[data["participant"] == 3]

participant_graph,translator_word_to_index = processing.convert_table_to_graph(
    complete_data_table=subdata,
    node_attr_names=["sc_liking","experience"],
    node_label_names=["sc_liking"],
    edge_attr_names=["senenceBERT_mpnet_similarity"],
    return_word_to_index=True)

print("participant_graph:",participant_graph)
print("participant_graph.x:",participant_graph.x)
print("participant_graph.edge_attr:",participant_graph.edge_attr)
print("participant_graph.y:",participant_graph.y)

Test function convert_table_to_graph
Data(x=[60, 2], edge_index=[2, 3540], edge_attr=[3540, 1], y=[60, 1], train_mask=[60], val_mask=[60])
validate: True
is undirected: True
has_self_loop: tensor(False)
end Test function convert_table_to_graph
participant_graph: Data(x=[60, 2], edge_index=[2, 3540], edge_attr=[3540, 1], y=[60, 1], train_mask=[60], val_mask=[60])
participant_graph.x: tensor([[ 1.2549e+00,  7.1000e+01],
        [ 1.2549e+00,  5.5000e+01],
        [ 1.2549e+00,  6.3000e+01],
        [ 7.0186e-01,  1.1000e+01],
        [-3.7343e-01,  0.0000e+00],
        [ 2.1030e-01,  2.5000e+01],
        [ 6.0970e-01,  2.5000e+01],
        [ 1.2549e+00,  1.0000e+02],
        [ 8.2476e-01,  2.1000e+01],
        [-7.4210e-01,  5.4000e+01],
        [ 5.1753e-01,  1.1000e+01],
        [ 1.2549e+00,  1.0000e+02],
        [-1.8174e+00,  7.3000e+01],
        [ 1.2549e+00,  1.0000e+02],
        [ 7.3259e-01,  6.1000e+01],
        [ 1.0091e+00,  8.6000e+01],
        [-8.6499e-01,  2.3000e+01],
  

/home/julien-rsbrg/Documents/UCL/SemanticToLiking/src/processing.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  extracted_features_word1.rename(columns=col_renaming,inplace=True)
/home/julien-rsbrg/Documents/UCL/SemanticToLiking/src/processing.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  extracted_features_word2.rename(columns=col_renaming,inplace=True)
/home/julien-rsbrg/Documents/UCL/SemanticToLiking/src/processing.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

In [5]:
from torch_geometric.utils import to_networkx
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np

import plotly.graph_objects as go

from typing import Optional, Iterable

def convert_float_to_hsl(l,min_l,max_l,s=100,h=235):
    if max_l < min_l:
        min_l, max_l = max_l, min_l
        print("Careful: min max lighting exchanged")

    if max_l != min_l:
        scaled_l = (l-min_l)/(max_l-min_l)*100
        scaled_l = min(int(scaled_l),90)
    else:
        scaled_l = 90
    
    return f"hsl({h:d},{s:d}%,{scaled_l:d}%)"


def convert_torch_to_networkx_graph(data: torch_geometric.data.Data, edge_attr_id: int = 0, to_undirected:bool|str = False):
    G = to_networkx(data, to_undirected=to_undirected)
    weights = {e_pair:float(data.edge_attr[e,edge_attr_id]) for e,e_pair in enumerate(G.edges())} 
    nx.set_edge_attributes(G, values = weights, name = 'weight')
    return G

def draw_torch_graph(data: torch_geometric.data.Data, 
               node_annotations: Optional[Iterable] = None,
               node_colors: Optional[Iterable] = None,
               node_color_label: Optional[str] = "",
               title: Optional[str] = ""):
    G = convert_torch_to_networkx_graph(data=data)
    draw_networkx_graph(G,
                        node_annotations=node_annotations,
                        node_colors=node_colors,
                        node_color_label=node_color_label,
                        title=title)

def draw_networkx_graph(
        G: nx.Graph, 
        node_annotations: Optional[Iterable] = None,
        node_colors: Optional[Iterable] = None,
        node_color_label: Optional[str] = "",
        title: Optional[str] = ""):
    
    # adapts: https://plotly.com/python/network-graphs/
    pos = nx.spring_layout(G, k=20*1/np.sqrt(G.number_of_nodes()), weight="weight",seed = 50, iterations=500)


    edge_attr = list(nx.get_edge_attributes(G,"weight").values())
    min_edge_attr,max_edge_attr = np.min(edge_attr),np.max(edge_attr)
    
    edge_traces = []
    for edge in G.edges():
        edge_x = []
        edge_y = []
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.append(x0)
        edge_x.append(x1)
        edge_y.append(y0)
        edge_y.append(y1)

        color = convert_float_to_hsl(float(G.get_edge_data(edge[0],edge[1])["weight"]),min_edge_attr,max_edge_attr)
        edge_trace = go.Scatter(
            x=edge_x, y=edge_y,
            line=dict(width=0.5,color=color),
            hoverinfo='none', #to change
            mode='lines'
        )
        edge_traces.append(edge_trace)
    

    node_x = []
    node_y = []
    for node in G.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)

    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers',
        hoverinfo='text',
        marker=dict(
            showscale=True,
            # colorscale options
            #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
            #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
            #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
            colorscale='YlGnBu',
            reversescale=True,
            color=node_colors,
            size=10,
            colorbar=dict(
                thickness=15,
                title=dict(
                    text=node_color_label, # to change
                    side='right'
                ),
                xanchor='left',
            ),
            line_width=2))
    
    # colors
    node_text = []
    for node in G.nodes():
        # node_adjacencies.append(len(adjacencies[1]))
        node_text.append(str(node_annotations[node])+f":{node_colors[node]:.2f}")

    # node_trace.marker.color = node_adjacencies
    node_trace.text = node_text

    # plot
    fig = go.Figure(data=edge_traces+[node_trace],
             layout=go.Layout(
                title=dict(
                    text=title,
                    font=dict(
                        size=16
                    )
                ),
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                )
    fig.show()

participant_id = 3 # should be before when create participant graph
draw_torch_graph(participant_graph,
           node_annotations=list(translator_word_to_index.keys()),
           node_colors = participant_graph.x[:,0].tolist(),
           node_color_label="Liking for activity",
           title=f"Semantic network of activities with liking for participant {participant_id}")

In [8]:
def add_invert_edge_attr(G:nx.Graph,edge_attr_name:str,new_edge_attr_name:str):
    new_G = G.copy()
    edge_attr = nx.get_edge_attributes(new_G,name=edge_attr_name)
    new_edge_attr = {k:1/(v+1e-6) for k,v in edge_attr.items()}
    nx.set_edge_attributes(new_G, values = new_edge_attr, name = new_edge_attr_name)
    return new_G

def add_opposite_edge_attr(G:nx.Graph,edge_attr_name:str,new_edge_attr_name:str):
    new_G = G.copy() 
    edge_attr = nx.get_edge_attributes(new_G,name=edge_attr_name)
    new_edge_attr = {k:-v for k,v in edge_attr.items()}
    nx.set_edge_attributes(G, values = new_edge_attr, name = new_edge_attr_name)
    return new_G

In [11]:
def cluster_nodes(G:nx.Graph, opposite_weight:bool = False, upper_quantile_removed:float=.8):
    """
    Cluster nodes with Kruskal's algorithm. 
    The greater the weight the less likely the two nodes are in the same component.
    """
    edge_attr_name = "weight" 
    if opposite_weight:
        new_G = add_opposite_edge_attr(G,"weight","opposite_weight")
        edge_attr_name = "opposite_weight"

    G_MST = nx.minimum_spanning_tree(new_G, weight=edge_attr_name)

    print(edge_attr_name,edge_attr_name)
    opposite_weight = list(nx.get_edge_attributes(G_MST,name=edge_attr_name).values())
    print("opp",opposite_weight)

    q = np.quantile(opposite_weight,upper_quantile_removed)
    for edge,edge_opp_weight in nx.get_edge_attributes(G_MST,edge_attr_name).items():
        if edge_opp_weight >= q:
            G_MST.remove_edge(edge[0],edge[1])

    connected_components_index = sorted(nx.connected_components(G_MST), key=len, reverse=True)
    return connected_components_index, G_MST

def draw_clustering(G:nx.Graph,
                    opposite_weight: bool = True,
                    upper_quantile_removed:float=0.8,
                    node_annotations: Optional[Iterable] = None,
                    node_colors: Optional[Iterable] = None,
                    node_color_label: Optional[str] = ""):
    connected_components_index, G_MST = cluster_nodes(G, opposite_weight = opposite_weight, upper_quantile_removed = upper_quantile_removed)

    words = list(translator_word_to_index.keys())
    for nodes_index in connected_components_index:
        new_component_words = []
        for id in nodes_index:
            new_component_words.append(words[id])
        print("Component's words =", new_component_words)

        n_nodes = len(nodes_index)
        if n_nodes > 1: 
            component_subgraph = G_MST.subgraph(nodes_index)
            weights = list(nx.get_edge_attributes(component_subgraph,"weight").values())
            weight_message = f"min weight = {np.min(weights):.3f}, max weights = {np.max(weights):.3f}"

            draw_networkx_graph(
                component_subgraph,
                node_annotations=node_annotations,
                node_colors = node_colors,
                node_color_label = node_color_label,
                title=f"Component with {n_nodes:d} nodes ("+ weight_message +")")

G_participant_graph = convert_torch_to_networkx_graph(participant_graph,to_undirected="upper")
draw_clustering(G_participant_graph,
                upper_quantile_removed=.7,
                node_annotations=list(translator_word_to_index.keys()),
                node_colors = participant_graph.x[:,0].tolist(),
                node_color_label = "Activity liking")

opposite_weight opposite_weight
opp []


IndexError: cannot do a non-empty take from an empty axes.